In [1]:
import pandas as pd
import numpy as np
import os
import xarray as xr
import rasterio
from glob import glob
import geopandas as gpd

#path_fire = r"C:\Users\anfel\OneDrive\Desktop\M2\prjt\data\algeria_tunisia.csv"
#df_fire = pd.read_csv(path_fire)
df_fire_clean = pd.read_parquet(r"C:\Users\anfel\OneDrive\Desktop\M2\prjt\local_dataset\dataset\fire_full.parquet", engine="fastparquet")

#df_fire_clean = df_fire[['longitude', 'latitude', 'type']].copy()

print(df_fire_clean.head())

        lon       lat  fire
0   9.48947  31.49290     1
1   9.49053  31.49524     1
2   9.49368  31.49449     1
3   9.49154  31.49420     1
4  10.09115  36.93407     1


In [2]:
print(f"Nombre de lignes dans df_fire_clean : {len(df_fire_clean)}")
print(f"Shape du dataset : {df_fire_clean.shape}")

Nombre de lignes dans df_fire_clean : 56864
Shape du dataset : (56864, 3)


In [3]:
# Créer l'attribut binaire 'fire'
# Si 'type' == 0, 'fire' = 1, sinon 'fire' = 0
"""df_fire_clean['fire'] = np.where(df_fire_clean['type'] == 0, 1, 0)

print(f"df_fire_clean avec colonne 'fire' créé : {df_fire_clean.shape}")
print(df_fire_clean.head())"""

'df_fire_clean[\'fire\'] = np.where(df_fire_clean[\'type\'] == 0, 1, 0)\n\nprint(f"df_fire_clean avec colonne \'fire\' créé : {df_fire_clean.shape}")\nprint(df_fire_clean.head())'

## Chargement et agrégation des données climatiques par saisons + Conversion en DataFrame

In [3]:
climate_csv_path = r"C:\Users\anfel\OneDrive\Desktop\M2\prjt\data\climat_clean.csv"
climate_df = pd.read_csv(climate_csv_path)

print("CSV climat chargé :", climate_df.shape)

# Convertir la date
climate_df["time"] = pd.to_datetime(climate_df["time"])

seasons_indices = {
    's1': [12, 1, 2],   # Hiver (Dec-Jan-Feb)
    's2': [3, 4, 5],    # Printemps
    's3': [6, 7, 8],    # Été
    's4': [9, 10, 11]   # Automne
}

# Ajouter colonne "season"

def get_season(date):
    m = date.month
    if m in [12, 1, 2]:
        return "s1"
    if m in [3, 4, 5]:
        return "s2"
    if m in [6, 7, 8]:
        return "s3"
    return "s4"

climate_df["season"] = climate_df["time"].apply(get_season)

all_dfs = []

variables = ["log_precip",  "tmax","amplitude_thermique"]

for var in variables:
    print(f"\nTraitement de la variable : {var}")

    for season in ["s1", "s2", "s3", "s4"]:
        
        df_season = climate_df[climate_df["season"] == season]

        if df_season.empty:
            print(f"   Saison {season} vide pour {var}")
            continue

        # Agrégation
        if var == "log_precip":
            df_agg = (
                df_season.groupby(["longitude", "latitude"])[var]
                .sum()
                .reset_index()
                .rename(columns={var: f"{var}_{season}"})
            )
        else:
            df_agg = (
                df_season.groupby(["longitude", "latitude"])[var]
                .mean()
                .reset_index()
                .rename(columns={var: f"{var}_{season}"})
            )

        all_dfs.append(df_agg)

        print(f"  → {var}_{season} agrégé ({df_agg.shape[0]} points)")


print("\n Toutes les variables saisonnières climatiques ont été générées.")

CSV climat chargé : (148292, 6)

Traitement de la variable : log_precip
  → log_precip_s1 agrégé (20301 points)
  → log_precip_s2 agrégé (20231 points)
  → log_precip_s3 agrégé (16200 points)

Traitement de la variable : log_precip
  → log_precip_s1 agrégé (20301 points)
  → log_precip_s2 agrégé (20231 points)
  → log_precip_s3 agrégé (16200 points)
  → log_precip_s4 agrégé (21540 points)

Traitement de la variable : tmax
  → tmax_s1 agrégé (20301 points)
  → tmax_s2 agrégé (20231 points)
  → tmax_s3 agrégé (16200 points)
  → tmax_s4 agrégé (21540 points)

Traitement de la variable : amplitude_thermique
  → amplitude_thermique_s1 agrégé (20301 points)
  → amplitude_thermique_s2 agrégé (20231 points)
  → log_precip_s4 agrégé (21540 points)

Traitement de la variable : tmax
  → tmax_s1 agrégé (20301 points)
  → tmax_s2 agrégé (20231 points)
  → tmax_s3 agrégé (16200 points)
  → tmax_s4 agrégé (21540 points)

Traitement de la variable : amplitude_thermique
  → amplitude_thermique_s1 agrég

## Fusion des DataFrames climatiques en un seul DataFrame

In [4]:
# Commencer avec le premier DataFrame
df_climat_merged = all_dfs[0]

# Fusionner tous les autres DataFrames sur longitude et latitude
for i, df in enumerate(all_dfs[1:], 1):
    df_climat_merged = pd.merge(df_climat_merged, df, on=['longitude', 'latitude'], how='inner')
    print(f"   → Fusion {i}/{len(all_dfs)-1} terminée")

print(f"\n DataFrame climatique fusionné créé : {df_climat_merged.shape}")
print(f" Colonnes : {list(df_climat_merged.columns)}")
print("\n Aperçu :")
print(df_climat_merged.head())

   → Fusion 1/11 terminée
   → Fusion 2/11 terminée
   → Fusion 3/11 terminée
   → Fusion 4/11 terminée
   → Fusion 5/11 terminée
   → Fusion 6/11 terminée
   → Fusion 7/11 terminée
   → Fusion 8/11 terminée
   → Fusion 9/11 terminée
   → Fusion 10/11 terminée
   → Fusion 11/11 terminée

 DataFrame climatique fusionné créé : (10113, 14)
 Colonnes : ['longitude', 'latitude', 'log_precip_s1', 'log_precip_s2', 'log_precip_s3', 'log_precip_s4', 'tmax_s1', 'tmax_s2', 'tmax_s3', 'tmax_s4', 'amplitude_thermique_s1', 'amplitude_thermique_s2', 'amplitude_thermique_s3', 'amplitude_thermique_s4']

 Aperçu :
   longitude   latitude  log_precip_s1  log_precip_s2  log_precip_s3  \
0     -8.625  27.291667       3.082025       0.916291       0.832909   
1     -8.625  27.375000       3.224341       0.916291       1.964311   
2     -8.625  27.458333       3.655782       1.321756       1.996060   
3     -8.625  27.541667       3.654403       1.267652       1.163151   
4     -8.625  27.625000       2.0149

## Merge final : Fire + Climat (approche optimisée avec pd.merge)

In [5]:
# Arrondir les coordonnées à la résolution de la grille climatique
resolution = 1/12  # 0.083333°

# Fonction d'arrondissement améliorée
def round_to_grid(value, resolution):
    return np.round(value / resolution) * resolution

# Créer des copies avec coordonnées arrondies POUR LES DEUX DATASETS
df_fire_for_merge = df_fire_clean.copy()
df_fire_for_merge['lon_rounded'] = round_to_grid(df_fire_clean['lon'], resolution)
df_fire_for_merge['lat_rounded'] = round_to_grid(df_fire_clean['lat'], resolution)

df_climat_for_merge = df_climat_merged.copy()
df_climat_for_merge['longitude_rounded'] = round_to_grid(df_climat_merged['longitude'], resolution)
df_climat_for_merge['latitude_rounded'] = round_to_grid(df_climat_merged['latitude'], resolution)

print(f"Coordonnées feu avant arrondi : lon={df_fire_clean['lon'].iloc[0]:.6f}, lat={df_fire_clean['lat'].iloc[0]:.6f}")
print(f"Coordonnées feu après arrondi : lon={df_fire_for_merge['lon_rounded'].iloc[0]:.6f}, lat={df_fire_for_merge['lat_rounded'].iloc[0]:.6f}")

# Merge direct avec les colonnes arrondies
df_final = pd.merge(
    df_fire_for_merge,
    df_climat_for_merge,
    left_on=['lon_rounded', 'lat_rounded'],
    right_on=['longitude_rounded', 'latitude_rounded'],
    how='left'
)

# Supprimer les colonnes temporaires
cols_to_drop = ['lon_rounded', 'lat_rounded', 'longitude_rounded', 'latitude_rounded', 'longitude', 'latitude']
df_final = df_final.drop(columns=[col for col in cols_to_drop if col in df_final.columns])

print(f"\nDataset final après merge : {df_final.shape}")
print(f"Valeurs manquantes dans les variables climatiques :")
climatic_cols = [col for col in df_final.columns if any(season in col for season in ['s1', 's2', 's3', 's4'])]
print(df_final[climatic_cols].isnull().sum())

# Vérifier le pourcentage de correspondances
matches_count = df_final[climatic_cols[0]].notna().sum()
total_points = len(df_final)
print(f"\nPoints avec données climatiques : {matches_count}/{total_points} ({matches_count/total_points*100:.1f}%)")

Coordonnées feu avant arrondi : lon=9.489470, lat=31.492900
Coordonnées feu après arrondi : lon=9.500000, lat=31.500000

Dataset final après merge : (60463, 15)
Valeurs manquantes dans les variables climatiques :
log_precip_s1             44155
log_precip_s2             44155
log_precip_s3             44155
log_precip_s4             44155
tmax_s1                   44155
tmax_s2                   44155
tmax_s3                   44155
tmax_s4                   44155
amplitude_thermique_s1    44155
amplitude_thermique_s2    44155
amplitude_thermique_s3    44155
amplitude_thermique_s4    44155
dtype: int64

Points avec données climatiques : 16308/60463 (27.0%)


In [6]:
# Charger landcover avec les polygones originaux
gdf_land = gpd.read_file(r"C:\Users\anfel\OneDrive\Desktop\M2\prjt\data\landcover_final_ML.gpkg")

print(f"Land cover chargé : {gdf_land.shape[0]} polygones")
print(f"CRS : {gdf_land.crs}")
print(f"Colonnes : {list(gdf_land.columns)}")

c:\Users\anfel\anaconda3\envs\dm\lib\site-packages\pyogrio\core.py:35: RuntimeWarning: Could not detect GDAL data files.  Set GDAL_DATA environment variable to the correct path.
  _init_gdal_data()


Land cover chargé : 307385 polygones
CRS : EPSG:4326
Colonnes : ['GRIDCODE', 'pays', 'log_area_sqm', 'lcc_code_encoded', 'geometry']


In [7]:
# Convertir df_final en GeoDataFrame avec des points géométriques
gdf_final = gpd.GeoDataFrame(
    df_final,
    geometry=gpd.points_from_xy(df_final['lon'], df_final['lat']),
    crs="EPSG:4326"  # WGS84 (système de coordonnées standard)
)

print(f"\nGeoDataFrame créé : {gdf_final.shape}")
print(f"CRS : {gdf_final.crs}")
print(f"Type de géométrie : {gdf_final.geometry.geom_type.unique()}")


GeoDataFrame créé : (60463, 16)
CRS : EPSG:4326
Type de géométrie : ['Point']


In [8]:
# Aligner les systèmes de coordonnées si nécessaire
if gdf_final.crs != gdf_land.crs:
    print(f"\nReprojection de gdf_land : {gdf_land.crs} → {gdf_final.crs}")
    gdf_land = gdf_land.to_crs(gdf_final.crs)
else:
    print(f"\nLes CRS sont déjà alignés : {gdf_final.crs}")


Les CRS sont déjà alignés : EPSG:4326


In [9]:
# 1. Vérifier les bounding boxes
print("\n Points (Fire + Climat):")
points_bounds = gdf_final.total_bounds
print(f"   Min Lon: {points_bounds[0]:.4f}, Max Lon: {points_bounds[2]:.4f}")
print(f"   Min Lat: {points_bounds[1]:.4f}, Max Lat: {points_bounds[3]:.4f}")

print("\n  Polygones (Land Cover):")
land_bounds = gdf_land.total_bounds
print(f"   Min Lon: {land_bounds[0]:.4f}, Max Lon: {land_bounds[2]:.4f}")
print(f"   Min Lat: {land_bounds[1]:.4f}, Max Lat: {land_bounds[3]:.4f}")

# 2. Vérifier si les zones se chevauchent
lon_overlap = not (points_bounds[2] < land_bounds[0] or points_bounds[0] > land_bounds[2])
lat_overlap = not (points_bounds[3] < land_bounds[1] or points_bounds[1] > land_bounds[3])

if lon_overlap and lat_overlap:
    print(" Les zones SE CHEVAUCHENT ")
else:
    print(" Les zones NE SE CHEVAUCHENT PAS ")
    if not lon_overlap:
        print(" Pas de chevauchement en LONGITUDE")
    if not lat_overlap:
        print("Pas de chevauchement en LATITUDE")

# 3. Échantillon de points
print("\n ÉCHANTILLON DE POINTS (5 premiers)")
print(gdf_final[['lon', 'lat', 'geometry']].head())

# 4. Échantillon de polygones
print("\n ÉCHANTILLON DE POLYGONES (5 premiers)")
print(gdf_land[['GRIDCODE', 'geometry']].head())

# 5. Vérifier les géométries invalides
print("\n VÉRIFICATION DE LA VALIDITÉ DES GÉOMÉTRIES")
invalid_points = (~gdf_final.is_valid).sum()
invalid_polys = (~gdf_land.is_valid).sum()
print(f"   Points invalides : {invalid_points}/{len(gdf_final)}")
print(f"   Polygones invalides : {invalid_polys}/{len(gdf_land)}")

if invalid_points > 0 or invalid_polys > 0:
    print("    Il y a des géométries invalides !")
else:
    print("    Toutes les géométries sont valides")


 Points (Fire + Climat):
   Min Lon: -8.1280, Max Lon: 11.1119
   Min Lat: 19.5932, Max Lat: 37.3332

  Polygones (Land Cover):
   Min Lon: -8.6722, Max Lon: 11.9681
   Min Lat: 18.9660, Max Lat: 37.5439
 Les zones SE CHEVAUCHENT 

 ÉCHANTILLON DE POINTS (5 premiers)
        lon       lat                   geometry
0   9.48947  31.49290    POINT (9.48947 31.4929)
1   9.49053  31.49524   POINT (9.49053 31.49524)
2   9.49368  31.49449   POINT (9.49368 31.49449)
3   9.49154  31.49420    POINT (9.49154 31.4942)
4  10.09115  36.93407  POINT (10.09115 36.93407)

 ÉCHANTILLON DE POLYGONES (5 premiers)
   GRIDCODE                                           geometry
0       210  POLYGON ((6.41528 37.08696, 6.43103 37.0855, 6...
1       210  POLYGON ((7.37137 37.08194, 7.3709 37.08717, 7...
2        50  POLYGON ((6.12361 36.68472, 6.12361 36.69306, ...
3       130  POLYGON ((6.44583 37.07917, 6.44583 37.08194, ...
4        50  POLYGON ((6.40694 37.08194, 6.40694 37.08472, ...

 VÉRIFICATION DE L

In [13]:
BUFFER_SIZE = 0.005  # 500 mètres

# Créer une copie avec buffer
gdf_final_buffered = gdf_final.copy()
gdf_final_buffered['geometry'] = gdf_final_buffered.geometry.buffer(BUFFER_SIZE)

# Spatial join sur tout le dataset
gdf_full = gpd.sjoin(
    gdf_final_buffered,
    gdf_land[['geometry', 'GRIDCODE', 'log_area_sqm', 'lcc_code_encoded']],
    how="left",
    predicate="intersects"
)

# Statistiques
total_rows = len(gdf_full)
unique_points = gdf_full.index.nunique()
matches = gdf_full['GRIDCODE'].notna().sum()
unique_matches = gdf_full[gdf_full['GRIDCODE'].notna()].index.nunique()


print(f"\n RÉSULTATS :")
print(f"   • Total de lignes après join : {total_rows:,}")
print(f"   • Points uniques traités : {unique_points:,}")
print(f"   • Lignes avec correspondance : {matches:,}")
print(f"   • Points uniques ayant trouvé un match : {unique_matches:,}")
print(f"   • Taux de couverture : {(unique_matches/unique_points)*100:.1f}%")

# Vérifier les duplications
duplications = total_rows - unique_points
if duplications > 0:
    print(f"\n  {duplications:,} lignes dupliquées détectées")
else:
    print(f"\n Aucune duplication ")

C:\Users\anfel\AppData\Local\Temp\ipykernel_8408\1355669867.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_final_buffered['geometry'] = gdf_final_buffered.geometry.buffer(BUFFER_SIZE)



 RÉSULTATS :
   • Total de lignes après join : 71,724
   • Points uniques traités : 60,463
   • Lignes avec correspondance : 31,934
   • Points uniques ayant trouvé un match : 20,673
   • Taux de couverture : 34.2%

  11,261 lignes dupliquées détectées


In [14]:
# Supprimer les duplications (garder la première correspondance pour chaque point)
if len(gdf_full) > len(gdf_final):
    print(f"   Avant dé-duplication : {len(gdf_full):,} lignes")
    gdf_full = gdf_full[~gdf_full.index.duplicated(keep='first')]
    print(f"   Après dé-duplication : {len(gdf_full):,} lignes")
    removed = len(gdf_full) - len(gdf_final)
    if removed != 0:
        print(f"Duplications supprimées")
else:
    print(" Aucune duplication à nettoyer")

# Nettoyage final : supprimer les colonnes inutiles
df_full = gdf_full.drop(columns=['geometry', 'index_right'], errors='ignore')
df_full = pd.DataFrame(df_full)

print(f"\nDATASET FINAL :")
print(f"   Shape : {df_full.shape}")
print(f"   Colonnes : {list(df_full.columns)}")

# Statistiques finales
land_cover_matches = df_full['GRIDCODE'].notna().sum()
total_points = len(df_full)
print(f"\n COUVERTURE LAND COVER :")
print(f"   Points avec land cover : {land_cover_matches:,}/{total_points:,} ({(land_cover_matches/total_points)*100:.1f}%)")
print(f"   Points sans land cover : {total_points - land_cover_matches:,} ({((total_points - land_cover_matches)/total_points)*100:.1f}%)")

print("\n" + "="*70)

   Avant dé-duplication : 71,724 lignes
   Après dé-duplication : 60,463 lignes

DATASET FINAL :
   Shape : (60463, 18)
   Colonnes : ['lon', 'lat', 'fire', 'log_precip_s1', 'log_precip_s2', 'log_precip_s3', 'log_precip_s4', 'tmax_s1', 'tmax_s2', 'tmax_s3', 'tmax_s4', 'amplitude_thermique_s1', 'amplitude_thermique_s2', 'amplitude_thermique_s3', 'amplitude_thermique_s4', 'GRIDCODE', 'log_area_sqm', 'lcc_code_encoded']

 COUVERTURE LAND COVER :
   Points avec land cover : 20,673/60,463 (34.2%)
   Points sans land cover : 39,790 (65.8%)



In [17]:
# Créer les centroïdes des polygones land cover
gdf_land_centroids = gdf_land.copy()
gdf_land_centroids['geometry'] = gdf_land_centroids.geometry.centroid

# sjoin_nearest trouve automatiquement le point le plus proche
gdf_full_centroids = gpd.sjoin_nearest(
    gdf_final,
    gdf_land_centroids[['geometry', 'GRIDCODE', 'log_area_sqm', 'lcc_code_encoded']],
    how='left',
    distance_col='distance_to_centroid_deg'
)

# Convertir la distance en mètres
gdf_full_centroids['distance_to_centroid_m'] = gdf_full_centroids['distance_to_centroid_deg'] * 111000

# Nettoyer les colonnes inutiles
df_full_centroids = gdf_full_centroids.drop(columns=['geometry', 'index_right'], errors='ignore')
df_full_centroids = pd.DataFrame(df_full_centroids)

print(f"\n RÉSULTATS AVEC CENTROÏDES :")
print(f"   • Dataset shape : {df_full_centroids.shape}")
print(f"   • Points avec land cover : {df_full_centroids['GRIDCODE'].notna().sum():,} ({(df_full_centroids['GRIDCODE'].notna().sum()/len(df_full_centroids))*100:.1f}%)")


C:\Users\anfel\AppData\Local\Temp\ipykernel_8408\2233567768.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_land_centroids['geometry'] = gdf_land_centroids.geometry.centroid
c:\Users\anfel\anaconda3\envs\dm\lib\site-packages\geopandas\array.py:408: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(
c:\Users\anfel\anaconda3\envs\dm\lib\site-packages\geopandas\array.py:408: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(



 RÉSULTATS AVEC CENTROÏDES :
   • Dataset shape : (60463, 20)
   • Points avec land cover : 60,463 (100.0%)


In [18]:
# Enregistrer le dataset final en format Parquet
output_path = r"C:\Users\anfel\OneDrive\Desktop\M2\prjt\local_dataset\dataset\buffer.parquet"

df_full_centroids.to_parquet(output_path, engine='fastparquet', compression='snappy')
